# 🤗 Database Setup

This notebook is used to upload the Huggingface Climate Policy Radar dataset to a Postgres table.

Refer to README.md for instructions on setting up the Postgres database.

In [1]:
import os
import regex as re
from tqdm.notebook import tqdm
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import pgai
import torch
import glob
from transformers import AutoTokenizer, AutoModel, AutoModelForMaskedLM
from datasets import load_dataset, Features, Value
from functions import generate_embeddings_for_text, store_database_batched
from sqlalchemy import create_engine, text

## 1. Load the Huggingface dataset.

In [4]:
# Login using e.g. `huggingface-cli login` in the command line to access this dataset

ds = load_dataset("ClimatePolicyRadar/all-document-text-data")
ds = ds["train"]
flat_ds = ds.flatten()

Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Loading dataset shards:   0%|          | 0/42 [00:00<?, ?it/s]

## 2. Save all chunks into the "climate_policy_radar" table.

The table contains all documents from the Climate Policy Radar dataset, with each row representing a chunk and all associated metadata stored in individual columns.

In [ ]:
store_database_batched(flat_ds, num_chunks=len(flat_ds))